In [1]:
import torch
import numpy as np
from collections import defaultdict

In [2]:
# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print("There are %d GPU(s) available." % torch.cuda.device_count())

    print("We will use the GPU:", torch.cuda.get_device_name(0))

# If not...
# elif torch.backends.mps.is_available():
#     device = torch.device("mps")

#     print("Using mps backend")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [3]:
import h5py
imdb = h5py.File('../data/imdb_1024.h5')
sgdb = h5py.File('../data/VG-SGG.h5')

import json
with open('../data/VG-SGG-dicts.json') as f:
    sgdicts = json.load(f)

In [4]:
triplets = defaultdict(set)
for i, s in enumerate(sgdb['relationships']):
    sub = sgdb['labels'][s[0]][0]
    obj = sgdb['labels'][s[1]][0]
    rel = sgdb['predicates'][i][0]
    triplets[(sub, obj)].add(rel)

len(triplets)

10873

In [5]:
from tensordict import TensorDict
subjects = []
objects = []
relations = []
for k in triplets:
    sub = torch.zeros(150)
    sub[k[0]-1] = 1
    obj = torch.zeros(150)
    obj[k[1]-1] = 1
    rels = torch.zeros(50)
    for rel in triplets[k]:
        rels[rel-1] = 1
    subjects.append(sub)
    objects.append(obj)
    relations.append(rels)


print(len(subjects))
print(len(objects))
print(len(relations))

subjects = torch.vstack(subjects)
objects = torch.vstack(objects)
relations = torch.vstack(relations)

print(subjects.shape)
print(objects.shape)
print(relations.shape)

10873
10873
10873
torch.Size([10873, 150])
torch.Size([10873, 150])
torch.Size([10873, 50])


In [6]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
dataset = TensorDataset(subjects, objects, relations)
batch_size = 32

train_dataloader = DataLoader(
            dataset,  # The training samples.
            sampler = RandomSampler(dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )


In [7]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(300, 150)
        self.dropout = torch.nn.Dropout(p=0.2)
        self.fc2 = torch.nn.Linear(150, 50)
        self.softmax = torch.nn.Softmax(dim=0)

    def forward(self, inputs1, inputs2):
        inputs = torch.cat((inputs1, inputs2), dim=1)
        inputs.to(device=device)
        x = self.fc1(inputs)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x
        
model = MLP()
model.to(device=device)

MLP(
  (fc1): Linear(in_features=300, out_features=150, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=150, out_features=50, bias=True)
  (softmax): Softmax(dim=0)
)

In [8]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-1)

In [9]:
for epoch in range(10):
    model.train()
    average_loss = 0
    for batch in train_dataloader:
        model.zero_grad()
        subjects_ = batch[0].to(device)
        objects_ = batch[1].to(device)
        target = batch[2].to(device)
        output = model(subjects_, objects_)
        loss = criterion(output, target)
        average_loss += loss.item()
        loss.backward()
        optimizer.step()
    average_loss = average_loss / len(train_dataloader)
    print('Epoch {epoch} ===> Loss: {avg_loss:.3f}'.format(epoch=epoch+1, avg_loss=average_loss))

Epoch 1 ===> Loss: 0.087
Epoch 2 ===> Loss: 0.089
Epoch 3 ===> Loss: 0.090
Epoch 4 ===> Loss: 0.090
Epoch 5 ===> Loss: 0.090
Epoch 6 ===> Loss: 0.090
Epoch 7 ===> Loss: 0.090
Epoch 8 ===> Loss: 0.090
Epoch 9 ===> Loss: 0.090
Epoch 10 ===> Loss: 0.091


In [10]:
subjects.shape

torch.Size([10873, 150])

In [11]:
model.eval()
avg_accuracy = 0
avg_recall = 0
avg_precision = 0
for i in range(100):
    subject = torch.reshape(subjects[i], (1, subjects[i].shape[0]))
    obj = torch.reshape(objects[i], (1, objects[i].shape[0]))
    subject.to(device=device)
    obj.to(device=device)
    output = model(subject, obj)[0]
    # print(output)
    target = relations[i]
    tp = 0
    tn = 0
    fp = 0
    fn = 0 
    for i in range(50):
        if target[i] == 1 and output[i] == 1:
            tp += 1
        elif target[i] == 1 and output[i] == 0:
            fn += 1
        elif target[i] == 0 and output[i] == 0:
            tn += 1
        else:
            fp += 1

    accuracy = tp/(tp+tn+fp+fn)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    avg_accuracy += accuracy
    avg_recall += recall
    avg_precision += precision
avg_accuracy /= 100
avg_recall /= 100
avg_precision /= 100

print('===== Avg. Accuracy: {accuracy:.2f} ======'.format(accuracy=avg_accuracy))
print('===== Avg. Recall: {recall} ======'.format(recall=avg_recall))
print('===== Avg. Precision: {precision:.2f} ======'.format(precision=avg_precision))

===== Avg. Accuracy: 0.16 ======
===== Avg. Recall: 1.0 ======
===== Avg. Precision: 0.16 ======
